In [ ]:
import pandas as pd
import xlwings as xw
from itertools import product
from pprint import pprint as pp

from sixgill.definitions import *
from sixgill.pipesim import Model, Units

In [ ]:
folderPAth = r"F:\Project\AET01083.00 Wave-Component_3A\09. Documents, Drawings\9.31 Process\09.31.01 Current documents\BAB Pump Hydraulics\1. HP Model"
fileName = "BAB HP_NETWORK Winter.pips"
modelPath = folderPAth + "\\" + fileName

###########################################

excel_file = "Water Injection Profile for BAB HP.xlsx"
excel_path = folderPAth + "\\" + excel_file

excel_df = pd.read_excel(excel_path, sheet_name="PIPSIM Input", header=3, index_col=0)
df_Conditions = pd.read_excel(
    excel_path, sheet_name="Conditions", header=2, index_col=0
)
well_list = excel_df.index.to_list()

In [ ]:
model = Model.open(modelPath)
ns = model.tasks.networksimulation

In [ ]:
dict_bc = ns.get_conditions()
df_bc = pd.DataFrame.from_dict(dict_bc)

In [ ]:
dict_overall = model.get_values(show_units=True)
df_overall = pd.DataFrame.from_dict(dict_overall)

In [ ]:
required_cols = ["Unit"] + well_list
df_overallNew = df_overall[required_cols]
df_overallNew = df_overallNew.dropna(how="all", axis=0, subset=required_cols[1:])

In [ ]:
df_overallNew.loc["IsActive"] = df_overallNew.loc["IsActive"].apply(
    lambda x: True if x == False else x
)
model.set_values(dict=df_overallNew[well_list].to_dict())

In [ ]:
newSaveAsPath = "test.pips"
model.save(newSaveAsPath)
model.tasks.networksimulation.reset_conditions()

In [ ]:
df_New_bc = pd.DataFrame.from_dict(ns.get_conditions())

In [ ]:
df_New_bc.loc["LiquidFlowRate", well_list] = df_New_bc.loc[
    "LiquidFlowRate", well_list
].apply(lambda flow: 0.001 if flow < 0.0001 else flow)

In [ ]:
ns.set_conditions(boundaries=df_New_bc.to_dict())

In [ ]:
system_variables = [
    SystemVariables.PRESSURE,
    SystemVariables.TEMPERATURE,
    SystemVariables.VOLUME_FLOWRATE_WATER_STOCKTANK,
]
profile_variables = [
    ProfileVariables.TEMPERATURE,
    ProfileVariables.PRESSURE,
    ProfileVariables.ELEVATION,
    ProfileVariables.TOTAL_DISTANCE,
]

In [ ]:
results = model.tasks.networksimulation.run(
    profile_variables=profile_variables, system_variables=system_variables,
)

In [1]:
model.close()

NameError: name 'model' is not defined

In [ ]:
# results = ns.run(system_variables=system_variables, profile_variables=profile_variables)
# dict_NodeResults = results.node
# df_NodeResults = pd.DataFrame.from_dict(dict_NodeResults)

In [ ]:
# Unit Conversion (BPD to m3/s)
conversion_factor = 1.840131e-6
cols_for_conversion = excel_df.columns.to_list()[1:]
excel_df[cols_for_conversion] = excel_df[cols_for_conversion] * conversion_factor

In [ ]:
case = "C3"
for well in excel_df["Case"]:
    if well in df_bc.columns:
        flowrate = excel_df.loc[excel_df["Case"] == well, [case]].values[0][0]
        df_bc[well]["LiquidFlowRate"] = flowrate

In [ ]:
updated_bc = df_bc.to_dict()
ns.set_conditions(boundaries=updated_bc)
updated_results = ns.run(
    system_variables=system_variables, profile_variables=profile_variables
)

In [ ]:
dict_New_NodeResults = updated_results.node
df_New_NodeResults = pd.DataFrame.from_dict(dict_New_NodeResults)

In [ ]:
# Unit Conversion in results
# Pressure Pa a to barg
df_New_NodeResults["Pressure"].iloc[0] = "bar g"
df_New_NodeResults["Pressure"].iloc[1:] = (
    df_New_NodeResults["Pressure"].iloc[1:] - 101325
) / 1e5

# Temperature K to C
df_New_NodeResults["Temperature"].iloc[0] = "deg C"
df_New_NodeResults["Temperature"].iloc[1:] = (
    df_New_NodeResults["Temperature"].iloc[1:] - 273.15
)

# Flowrate m3/s to BPD
df_New_NodeResults["VolumeFlowrateWaterStockTank"].iloc[0] = "STBPD"
df_New_NodeResults["VolumeFlowrateWaterStockTank"].iloc[1:] = (
    df_New_NodeResults["VolumeFlowrateWaterStockTank"].iloc[1:] * conversion_factor
)

In [ ]:
wb = xw.Book(excel_path)
sheet_name = case + "_Results"
if sheet_name not in [sheet.name for sheet in wb.sheets]:
    wb.sheets.add(sheet_name)
ws = wb.sheets(sheet_name)
ws.clear_contents()

# ws.range("B2").value = df_NodeResults

ws.range("B2").value = df_New_NodeResults

new_file = folderPAth + "\\" + case + "_" + fileName
model.save(new_file)

model.close()

In [ ]:
model.close()